In [1]:
import math as m
import cmath as cm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib qt

# НАБОР ПАРАМЕТРОВ ИЗЛУЧЕНИЯ
wp = 0.0001 # ширина пучках в м
L = 0.0127 # длина кристалла в м
CLight = 3*10**8 # скорость света в вакууме в м/с   
f_1=5.826869931972789*10**14 # частота волны накачки в Гц
f_thz = 1. * 10**12 # частота ТГц-вой волны в Гц
f_2 = f_1 - f_thz # частота сигнальной волны в Гц

# ПОКАЗАТЕЛИ ПРЕЛОМЛЕНИЯ И КОЭФ ПОГЛОЩЕНИЯ
def NOptE(f): # показатель преломления необыкновенной оптической волны по ф-ле Селмейера, f - частота в Гц
    AOptE = 4.5401 # коэф-ты из формулы Селмейера для nе оптического
    BOptE = 9.6132*10**(4)
    COptE = 4.3931*10**(4)
    DOptE = 2.3567*10**(-8)
    x = m.sqrt(AOptE - BOptE/(COptE - (CLight/f)**2*10**18)- DOptE*(CLight/f)**2*10**18)
    return(x)
def gammaEffTHzE(f): # коэф для рассчёта диэл проницаемости необыкновенной ТГц волны, f - частота в Гц
    K3THzE = 3.241 * 10**48
    DeltaTHzE = 0.55 * 10**12
    f3THzE = 3.45 * 10**12
    gamma0THzE = 0.54 * 10**12
    gamma3THzE = 1.35 * 10**12
    tayTHzE = 0.6 * 10**(-12)
    x = gamma0THzE - (1j/f)*((K3THzE/(f3THzE**2 - f**2 - 1j*f*gamma3THzE)) + (DeltaTHzE**2/(1-1j*2*np.pi*f*tayTHzE)))
    return (x)
def ETHzE(f): # диэл проницаемость необыкновенной ТГц волны, f - частота в Гц
    STHzE = 26
    f0THzE = 7.44 * 10**12
    x = STHzE*f0THzE**2/(f0THzE**2 - f**2 - 1j*gammaEffTHzE(f)*f)
    return (x)
def NTHzE(f): # показатель преломления необыкновенной ТГц волны, f - частота в Гц
    x = cm.sqrt(ETHzE(f))
    return (x.real)
def alpha(f):  # коэф поглощения необыкновенной ТГц волны, f - частота в Гц
    x = cm.sqrt(ETHzE(f))*4*m.pi*f/CLight
    return (x.imag)
def my(f, theta):  # фактор пропорц коэф поглощения ТГц волны и учитывающий разные пути распростран
    return (alpha(f)/(2*m.cos(theta)))
n_thz = NTHzE(f_thz) # показ преломления ТГц волны в ниобате 
n_1 = NOptE(f_1) # показ преломления оптической волны на частоте f_1 в ниобате
n_2 = NOptE(f_2) # показ преломления оптической волны на частоте f_2 в ниобате
n_thz_si = 3.417 # показ преломления ТГц волны в кремние
n_thz_air = 1. # показ преломления ТГц волны в воздухе

# ВОЛНОВЫЕ ЧИСЛА
def k_thz_func(f):  # волновое число ТГц-овой волны в м^(-1)
    return (2 * np.pi * f * NTHzE(f)/CLight)
def k_opt_func(f): # волновое число оптической волны в м^(-1)
    return (2 * np.pi * f * NOptE(f)/CLight)
k_thz = k_thz_func(f_thz) # волновое число ТГц волны в ниобате
k_1 = k_opt_func(f_1) # волновое число оптической волны на частоте f_1 в ниобате
k_2 = k_opt_func(f_2) # волновое число оптической волны на частоте f_2 в ниобате
theta_0 = m.acos((n_1*f_1 - n_2*f_2)/(n_thz*f_thz)) # тета центра ТГц пучка в рад (из условия продольного синхронизма) 61.46752979771666
print(m.degrees(theta_0))

# УГЛОВАЯ СЕТКА В КРИСТАЛЛЕ
N_theta = 1000 # количество углов по тета в сетке для кристалла
N_phi = 1000  # количество углов по фи в сетке для кристалла

theta_min = m.radians(59.5)
theta_max = m.radians(63.5)
theta_arr = np.linspace(theta_min, theta_max, N_theta) # список уголов тета в рад
phi_arr = np.linspace(-m.pi/2, m.pi/2, N_phi) # список уголов фи в рад
'''
# ИЗНАЧАЛЬНАЯ ИНТЕНСИВНОСТЬ
def dkx(theta_thz, phi_thz, theta_2, phi_2):  # x-расстройка волнового вектора ТГц-вой волны (продольная)
    A = (k_2*m.sin(theta_2)*m.sin(phi_2) + k_thz*m.sin(theta_thz)*m.sin(phi_thz))/k_1
    B = (k_2*m.sin(theta_2)*m.cos(phi_2) + k_thz*m.sin(theta_thz)*m.cos(phi_thz))/k_1
    if (A**2 + B**2)<=1: # проверка существования угла
        cos_theta_1 = m.sqrt(1-(A**2 + B**2))
        return k_2*m.cos(theta_2)+k_thz*m.cos(theta_thz)-k_1*cos_theta_1
    else:
        return None

def integral(theta, phi): # расчёт изначальной интенсивности
    y0 =  np.linspace(0.0, m.radians(3.), 20)   #tetta
    x0 =  np.linspace(0.0, 2*m.pi, 90)   #phi
    V=0.+0.j
    V0 = 0.
    for y in y0:
       for x in x0:
            dkx_val = dkx(theta, phi, y, x)
            if dkx_val!=None:
                q = L/2. * (my(f_thz, theta) + 1j*dkx_val)
                V += m.sin(y) * cm.sinh(q)/q*cm.exp(-q)*cm.exp(-wp**2/4*(2*k_2**2 * m.sin(y)**2 + k_thz**2 * m.sin(theta)**2 + 2*k_2*k_thz*m.sin(theta)*m.sin(y)*m.cos(x-phi)))
            V0 += V.real**2 + V.imag**2
            V = 0.+0.j
    C = f_thz*m.pi/m.sqrt(n_thz)*m.sqrt(1.+3.*m.cos(phi)**2) # коэффициент перед интегралом
    return C**2*V0*wp**8

# РАСЧЁТ ИЗНАЧАЛЬНОЙ ИНТЕНСВНОСТИ
result = np.zeros((N_theta, N_phi), dtype=np.float64) # ТГц поле в кристалле

for i in range(0, N_theta):
    print(f"{m.degrees(theta_arr[i])}")
    for j in range(0, N_phi):
            y = theta_arr[i]
            x = phi_arr[j]
            result[i, j] = integral(y, x)

plt.pcolormesh(np.degrees(theta_arr), np.degrees(phi_arr), np.transpose(result), cmap='plasma')
plt.colorbar()
plt.ylabel('phi')
plt.xlabel('theta')
plt.show()
'''


# УГЛОВАЯ СЕТКА В КРЕМНИЕ
phi_si_max = np.arcsin(m.sqrt(1-(n_thz/n_thz_si)**2*m.cos(theta_min)**2)/(n_thz/n_thz_si*m.sin(theta_min)))
def nearest(lst, target):
    return min(lst, key=lambda x: abs(x-target))
N_theta_si = N_theta
N_phi_si_min = np.where(phi_arr == nearest(phi_arr, -phi_si_max))[0][0]+1
N_phi_si_max = np.where(phi_arr == nearest(phi_arr, phi_si_max))[0][0]-1
N_phi_si = N_phi_si_max - N_phi_si_min + 1
print(N_phi_si)


theta_arr_si = np.zeros((N_theta_si, N_phi_si), dtype=np.float64) # список уголов тета в рад
phi_arr_si = np.zeros((N_theta_si, N_phi_si), dtype=np.float64) # список уголов фи в рад

# ФОРМУЛЫ ФРЕНЕЛЯ
def alpha_1(n, k):
    return np.arccos(np.dot(n, k))
def alpha_2(alpha1, n12):
    if np.abs(n12*m.sin(alpha1))>1:
        return None
    return np.arcsin(n12*m.sin(alpha1))
def fresnel_s_intensity(n, k, n1, n2):
    alpha1 = alpha_1(n, k)
    n12 = n1/n2
    alpha2 = alpha_2(alpha1, n12)
    if alpha2==None:
        return 0.
    x = 1.-m.sin(alpha1-alpha2)**2/m.sin(alpha1+alpha2)**2
    #x = m.sin(alpha1)**2+m.sin(alpha1-alpha2)/m.sin(alpha1+alpha2)**2
    return x
def fresnel_p_intensity(n, k, n1, n2):
    alpha1 = alpha_1(n, k)
    n12 = n1/n2
    alpha2 = alpha_2(alpha1, n12)
    if alpha2==None:
        return 0.
    x = 1.-m.tan(alpha1-alpha2)**2/m.tan(alpha1+alpha2)**2
    #x = m.sin(alpha1)**2+m.sin(alpha1-alpha2)/(m.sin(alpha1+alpha2)**2+m.cos(alpha1-alpha2)**2)
    return x
def fresnel_s_amplitude(n, k, n1, n2):
    alpha1 = alpha_1(n, k)
    n12 = n1/n2
    alpha2 = alpha_2(alpha1, n12)
    if alpha2==None:
        return 0.
    x = 2*m.sin(alpha2)*m.cos(alpha1)/m.sin(alpha1+alpha2)
    return x
def fresnel_p_amplitude(n, k, n1, n2):
    alpha1 = alpha_1(n, k)
    n12 = n1/n2
    alpha2 = alpha_2(alpha1, n12)
    if alpha2==None:
        return 0.
    x = 2*m.sin(alpha2)*m.cos(alpha1)/(m.sin(alpha1+alpha2)*m.cos(alpha1-alpha2))
    return x

# УГЛЫ В КРЕМНИЕ
def Theta_si(theta, phi):
    n12 = n_thz/n_thz_si
    return np.arccos(n12*m.cos(theta))
def Phi_si(theta, phi):
    n12 = n_thz/n_thz_si
    y = 1 - n12**2*(m.cos(theta)**2+m.sin(phi)**2*m.sin(theta)**2)
    #y = 1 - n12**2*m.cos(theta)**2
    if y<0:
        return None
    return np.arctan(n12*m.sin(phi)*m.sin(theta)/m.sqrt(y))

def k_spherical(theta, phi): # волновой вектор в декартовых координатах через сферические
    return np.array([m.cos(theta), m.sin(theta)*m.cos(phi), m.sin(theta)*m.sin(phi)])

# ДОЛЯ S И P ПОЛЯРИЗОВАННОГО ИЗЛУЧЕНИЯ
def s_polarisation(theta, phi):
    x = m.cos(theta)**2/((1-m.sin(phi)**2*m.sin(theta)**2)*(1-m.cos(phi)**2*m.sin(theta)**2))
    return x

def p_polarisation(theta, phi):
    s = s_polarisation(theta, phi)
    return 1-s

# ОРТЫ ПОЛЯРИЗАЦИИ ПРИ ПЕРЕХОДЕ В КРЕМНИЙ
def e_s(theta, phi):
    e_s = np.array([m.tan(theta)*m.sin(phi), 0., -1.])
    return e_s/np.linalg.norm(e_s)

def e_p(theta, phi):
    theta_si = Theta_si(theta, phi)
    phi_si = Phi_si(theta, phi)
    if phi_si==None:
        return 0
    k_si = k_spherical(theta_si, phi_si)
    e_p = np.cross(k_si, e_s(theta, phi))
    return e_p/np.linalg.norm(e_p)
'''
def Jacobian_si(theta, phi):
    J = np.zeros((N_theta_si, N_phi_si), dtype=np.float64)
    hy = theta[1]-theta[0]
    hx = phi[1]-phi[0]
    d11, d22, d12, d21 = 0., 0., 0., 0.
    for i in range(0, N_theta_si):
        for j in range(0, N_phi_si):
            if i == 0:
                d11 = (Theta_si(theta[i+1], phi[j])-Theta_si(theta[i], phi[j]))/hy
                d21 = (Phi_si(theta[i+1], phi[j])-Phi_si(theta[i], phi[j]))/hy
            elif i==N_theta_si-1:
                d11 = (Theta_si(theta[i], phi[j])-Theta_si(theta[i-1], phi[j]))/hy
                d21 = (Phi_si(theta[i], phi[j])-Phi_si(theta[i-1], phi[j]))/hy
            else:
                d11 = (Theta_si(theta[i+1], phi[j])-Theta_si(theta[i-1], phi[j]))/(2*hy)
                d21 = (Phi_si(theta[i+1], phi[j])-Phi_si(theta[i-1], phi[j]))/(2*hy)
            if j == 0:
                d12 = (Theta_si(theta[i], phi[j+1])-Theta_si(theta[i], phi[j]))/hx
                d22 = (Phi_si(theta[i], phi[j+1])-Phi_si(theta[i], phi[j]))/hx
            elif j==N_phi_si-1:
                d12 = (Theta_si(theta[i], phi[j])-Theta_si(theta[i], phi[j-1]))/hx
                d22 = (Phi_si(theta[i], phi[j])-Phi_si(theta[i], phi[j-1]))/hx
            else:
                d12 = (Theta_si(theta[i], phi[j+1])-Theta_si(theta[i], phi[j-1]))/(2*hx)
                d22 = (Phi_si(theta[i], phi[j+1])-Phi_si(theta[i], phi[j-1]))/(2*hx)
            J[i, j] = d11*d22-d12*d22
    return J

jacobian_si = Jacobian_si(theta_arr[:], phi_arr[N_phi_si_min:N_phi_si_max+1])'''

'''def jacobian_si(theta_thz, phi_thz):
    n = n_thz/n_thz_si
    y = 1 - n**2*(m.cos(theta_thz)**2+m.sin(phi_thz)**2*m.sin(theta_thz)**2)
    if y<0:
        return 0
    else:
        y = m.sqrt(y)
        x = y**3/(n**2*m.sin(theta_thz)**2*m.cos(phi_thz)*(1-n**2*m.cos(theta_thz)**2))
        return m.fabs(x)'''


# РАСЧЁТ ИНТЕНСИВНОСТИ В КРЕМНИЕ
s_polarised_si = np.zeros((N_theta_si, N_phi_si), dtype=np.float64) # s-поляризованное ТГц поле в призме
p_polarised_si = np.zeros((N_theta_si, N_phi_si), dtype=np.float64) # p-поляризованное ТГц поле в призме
result_si = np.zeros((N_theta_si, N_phi_si), dtype=np.float64) # ТГц поле в призме
e_si =  np.zeros((3, N_theta, N_phi_si), dtype=np.float64) # орт поляризации ТГц поля в призме
'''
for i in range(0, N_theta_si):
    print(f"{m.degrees(theta_arr[i])}")
    for j in range(0, N_phi_si):
        y = theta_arr[i]
        x = phi_arr[j+N_phi_si_min]
        s = s_polarisation(y, x)
        p = p_polarisation(y, x)
        theta_arr_si[i, j] = Theta_si(y, x)
        phi_arr_si[i, j] = Phi_si(y, x)
        k = k_spherical(y, x)
        normal = np.array([0., 1., 0.])
        s_polarised_si[i, j] = result[i, j+N_phi_si_min]*fresnel_s_intensity(normal, k, n_thz, n_thz_si)*s#*jacobian_si(y, x)
        p_polarised_si[i, j] = result[i, j+N_phi_si_min]*fresnel_p_intensity(normal, k, n_thz, n_thz_si)*p#*jacobian_si(y, x)
        result_si[i, j] = s_polarised_si[i, j] + p_polarised_si[i, j]
        e_si_nenorm = e_s(y, x)*m.sqrt(s)*fresnel_s_amplitude(normal, k, n_thz, n_thz_si) + e_p(y, x)*m.sqrt(p)*fresnel_p_amplitude(normal, k, n_thz, n_thz_si)
        e_si[:, i, j] = e_si_nenorm/np.linalg.norm(e_si_nenorm)
'''
result_si = np.load('with_si.npy')
e_si = np.load('e_si.npy')
s_polarised_si  = np.load('s_si.npy')
p_polarised_si = np.load('p_si.npy')
theta_arr_si = np.load('theta_si.npy')
phi_arr_si = np.load('phi_si.npy')
#np.save('with_si', result_si)
#np.save('e_si', e_si)
#np.save('s_si', s_polarised_si)
#np.save('p_si', p_polarised_si)
#np.save('theta_si', theta_arr_si)
#np.save('phi_si', phi_arr_si)
'''
# ГРАФИКИ В КРЕМНИЕ
ax = plt.axes()
ax.set_facecolor("#0c0887")
plt.pcolormesh(np.transpose(np.degrees(theta_arr_si)), np.transpose(np.degrees(phi_arr_si)), np.transpose(result_si), cmap='plasma')
plt.colorbar()
plt.ylim(-60, +60)
plt.ylabel('phi')
plt.xlabel('theta')
plt.show()

ax = plt.axes()
ax.set_facecolor("#0c0887")
plt.pcolormesh(np.transpose(np.degrees(theta_arr_si)), np.transpose(np.degrees(phi_arr_si)), np.transpose(s_polarised_si), cmap='plasma')
plt.colorbar()
plt.ylabel('\phi')
plt.xlabel('\theta')
plt.show()

ax = plt.axes()
ax.set_facecolor("#0c0887")
plt.pcolormesh(np.transpose(np.degrees(theta_arr_si)), np.transpose(np.degrees(phi_arr_si)), np.transpose(p_polarised_si), cmap='plasma')
plt.colorbar()
plt.ylabel('phi')
plt.xlabel('theta')
plt.show()
'''



def incidence(center, source, k): # координаты точки падения
    b = np.dot(k, center-source)
    c = np.linalg.norm(center-source)**2-R**2
    r = b + m.sqrt(b**2-c)
    return r*k+source

def normal(theta, phi): # нормаль к границе раздела
    k = k_spherical(theta, phi)
    incident = incidence(center, source, k)
    n = incident - center
    n = n/np.linalg.norm(n)
    return n

def normal1(theta, phi): # нормаль к плоскости падения
    k = k_spherical(theta, phi)
    n1 = np.cross(k, normal(y, x))
    n1 = n1/np.linalg.norm(n1)
    return n1

# УГЛЫ В ВОЗДУХЕ
def Phi_air(theta, phi):
    k = k_spherical(theta, phi)
    n1 = normal1(theta, phi)
    n = normal(y, x)
    alpha1 = alpha_1(n, k)
    n12 = n_thz_si/n_thz_air
    alpha2 = alpha_2(alpha1, n12)
    if alpha2==None:
        return None
    A = (n[0]*n1[2]-n[2]*n1[0])/m.cos(alpha1)
    B = (n[0]*n1[1]-n[1]*n1[0])/m.cos(alpha2)
    C = n1[1]**2-n1[2]**2+A**2-B**2
    D = n1[1]*n1[2]-A*B
    E = n1[0]**2+n1[2]**2-A**2
    u = C**2+4*D**2
    v = 2*D**2-C*E
    w = E**2
    if v**2-u*w<0:
        print("Error: D<0")
        return None
    if phi > 0:
        phi_a = np.arccos(m.sqrt(1/u*(v+m.sqrt(v**2-u*w))))
    else:
        phi_a = -np.arccos(m.sqrt(1/u*(v+m.sqrt(v**2-u*w))))
    return phi_a

def Theta_air(theta, phi):
    n1 = normal1(theta, phi)
    phi_air = Phi_air(theta, phi)
    if phi_air == None:
        return None
    elif m.fabs(m.cos(phi_air)*n1[1]+m.sin(phi_air)*n1[2])<1.e-6:
        return None
    theta_a = np.arctan(-n1[0]/(m.cos(phi_air)*n1[1]+m.sin(phi_air)*n1[2]))
    return theta_a


def Jacobian_air(theta, phi, hx, hy):
    J = np.zeros((N_theta_si, N_phi_si), dtype=np.float64)
    d11, d22, d12, d21 = 0., 0., 0., 0.
    for i in range(0, N_theta_si):
        for j in range(0, N_phi_si):
            if i == 0:
                d11 = (theta[i+1, j]-theta[i, j])/(hx)
                d21 = (phi[i+1, j]-phi[i, j])/(hx)
            elif i==N_theta_si-1:
                d11 = (theta[i, j]-theta[i-1, j])/(hx)
                d21 = (phi[i, j]-phi[i-1, j])/(hx)
            else:
                d11 = (theta[i+1, j]-theta[i-1, j])/(2*hx)
                d21 = (phi[i+1, j]-phi[i-1, j])/(2*hx)
            if j == 0:
                d12 = (theta[i, j+1]-theta[i, j])/(hy)
                d22 = (phi[i, j+1]-phi[i, j])/(hy)
            elif j==N_phi_si-1:
                d12 = (theta[i, j]-theta[i, j-1])/(hy)
                d22 = (phi[i, j]-phi[i, j-1])/(hy)
            else:
                d12 = (theta[i, j+1]-theta[i, j-1])/(2*hy)
                d22 = (phi[i, j+1]-phi[i, j-1])/(2*hy)
            J[i, j] = d11*d22-d12*d21
    return J

def Jacobian_airN(theta, phi, hx, hy, N):
    J = np.zeros((N_theta_si, N), dtype=np.float64)
    d11, d22, d12, d21 = 0., 0., 0., 0.
    for i in range(0, N_theta_si):
        for j in range(0, N):
            if i == 0:
                d11 = (theta[i+1, j]-theta[i, j])/(hx)
                d21 = (phi[i+1, j]-phi[i, j])/(hx)
            elif i==N_theta_si-1:
                d11 = (theta[i, j]-theta[i-1, j])/(hx)
                d21 = (phi[i, j]-phi[i-1, j])/(hx)
            else:
                d11 = (theta[i+1, j]-theta[i-1, j])/(2*hx)
                d21 = (phi[i+1, j]-phi[i-1, j])/(2*hx)
            if j == 0:
                d12 = (theta[i, j+1]-theta[i, j])/(hy)
                d22 = (phi[i, j+1]-phi[i, j])/(hy)
            elif j==N-1:
                d12 = (theta[i, j]-theta[i, j-1])/(hy)
                d22 = (phi[i, j]-phi[i, j-1])/(hy)
            else:
                d12 = (theta[i, j+1]-theta[i, j-1])/(2*hy)
                d22 = (phi[i, j+1]-phi[i, j-1])/(2*hy)
            J[i, j] = d11*d22-d12*d21
    return J

# РАСЧЁТ ИЗЛУЧЕНИЯ В ВОЗДУХЕ
s_polarised_air = np.zeros((N_theta_si, N_phi_si), dtype=np.float64) # s-поляризованное ТГц поле в воздухе
p_polarised_air = np.zeros((N_theta_si, N_phi_si), dtype=np.float64) # p-поляризованное ТГц поле в воздухе
result_air = np.zeros((N_theta_si, N_phi_si), dtype=np.float64) # ТГц поле в воздухе
theta_arr_air = np.zeros((N_theta_si, N_phi_si), dtype=np.float64)
phi_arr_air = np.zeros((N_theta_si, N_phi_si), dtype=np.float64)

for e1 in [0.1]:
    e1 = - 0.52080381321156 + e1
    # ПАРАМЕТРЫ ЛИНЗЫ  ПРИЗМЫ
    '''D = 0.0104 # диаметр линзы в м
    R = 0.006 # радиус кривизны линзы в м
    t = 0.003 + e1*R # толщина линзы в м'''
    D = 0.0115 # диаметр линзы в м
    R = 0.006 # радиус кривизны линзы в м
    t = 0.0043 + e1*R # толщина линзы в м
    #delta = m.radians(46.1) # угол при вершине призмы
    delta = m.radians(40)
    
    x0 = L - D/2./m.cos(delta) # x координата источника
    a = D/2.*m.tan(delta)-(R-t) # расстояние между источником и центром кривизны линзы

    center = np.array([x0+a*m.sin(delta), a*m.cos(delta), 0]) # координаты центра линзы
    source = np.array([x0, 0., 0.]) # координаты источника

    dzeta = a/R

    for i in range(0, N_theta_si):
        #print(f"{m.degrees(theta_arr[i])}")
        for j in range(0, N_phi_si):
            y = theta_arr_si[i, j]
            x = phi_arr_si[i, j]
            theta_arr_air[i, j] = Theta_air(y, x)
            '''if m.isnan(theta_arr_air[i, j]):
                 theta_arr_air[i, j] =  0 '''               
            phi_arr_air[i, j] = Phi_air(y, x)
            '''if m.isnan(phai_arr_air[i, j]):
                 phi_arr_air[i, j] =  0'''
            k = k_spherical(y, x) # волновой вектор
            n = normal(y, x)
            n1 = normal1(y, x) # нормаль к плоскости падения
            s_polarised_si[i, j] = result_si[i, j]*np.dot(n1, e_si[:, i, j])**2
            p_polarised_si[i, j] = result_si[i, j]*(1-np.dot(n1, e_si[:, i, j])**2)
            s_polarised_air[i, j] = s_polarised_si[i, j]*fresnel_s_intensity(n, k, n_thz_si, n_thz_air)#*jacobian_air[i, j]
            p_polarised_air[i, j] = p_polarised_si[i, j]*fresnel_p_intensity(n, k, n_thz_si, n_thz_air)#*jacobian_air[i, j]
            result_air[i, j] = (s_polarised_air[i, j]+p_polarised_air[i, j])*10**29

    N = 0
    i = 0
    power = 0
    for x in theta_arr_air[0, 0:int(N_phi_si/2)]:
        if m.isnan(x):
            N+= 1
        i = i + 1
    #print(N, N_phi_si-2*N)
    hy = theta_arr[1]-theta_arr[0]
    hx = phi_arr[1]-phi_arr[0]
    jacobian_air = Jacobian_air(theta_arr_air, phi_arr_air, hx, hy)

    '''for i in range(0, N_theta_si):
        for j in range(0, N_phi_si):
            result_air[i, j] /= m.fabs(jacobian_air[i, j])'''
    N=0
    for x in theta_arr_air[0, 0:int(N_phi_si/2)]:
        if m.isnan(x):
            N+= 1
    #print(N, N_phi_si-2*N)

    hy = theta_arr[1]-theta_arr[0]
    hx = phi_arr[1]-phi_arr[0]
    #print('hx=', hx, 'hy=', hy)
    if N == 0:
        jacobian_air = Jacobian_air(theta_arr_air, phi_arr_air, hx, hy)
    else:
        jacobian_air = Jacobian_airN(theta_arr_air[:, N:-N], phi_arr_air[:, N:-N], hx, hy, N_phi_si-2*N)
    
    for i in range(0, N_theta_si):
        for j in range(0, N_phi_si-2*N):
            result_air[i, j+N] /= m.fabs(jacobian_air[i, j])
            if phi_arr_air[i, j+N] < m.radians(5) and phi_arr_air[i, j+N]>m.radians(-5):
                if theta_arr_air[i, j+N] < m.radians(48.9) and theta_arr_air[i, j+N] > m.radians(38.9):
                    power = power + result_air[i, j+N]
    print('dzeta =', dzeta, 'power =', power)
# ГРАФИКИ В ВОЗДУХЕ

ax = plt.axes()
ax.set_facecolor("#0c0887")
plt.pcolormesh(np.transpose(np.degrees(theta_arr_air)), np.transpose(np.degrees(phi_arr_air)), np.transpose(result_air), cmap='plasma')
plt.colorbar()
plt.ylabel('phi')
plt.ylim(-20, 20)
plt.xlim(44, 49)
plt.xlabel('theta')
plt.title('I_THz')
plt.show()


<>:287: SyntaxWarning: invalid escape sequence '\p'
<>:287: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Ahito\AppData\Local\Temp\ipykernel_7600\3191567995.py:287: SyntaxWarning: invalid escape sequence '\p'
  '''
C:\Users\Ahito\AppData\Local\Temp\ipykernel_7600\3191567995.py:287: SyntaxWarning: invalid escape sequence '\p'
  '''


ModuleNotFoundError: No module named 'numpy'